This code uses the MH-Sensor-series Flying Fish module
that has an LDR wired to an amplifier with a gain pot 
to turn the brightness of a light into a digital signal.
http://musuk.guru/blog/flying-fish-MH-sensor-series

This is best used to detect flashes of an LED on an electricity 
meter and send the signals of each 1/1000th of a kWh through 
mqtt to the DoESLiverpool network.

Uses function: machine.time_pulse_us(pin, pulse_level, timeout_us)
>    Time a pulse on the given pin, and return the duration of the pulse in microseconds. The pulse_level argument should be 0 to time a low pulse or 1 to time a high pulse.

>    If the current input value of the pin is different to pulse_level, the function first (%) waits until the pin input becomes equal to pulse_level, then (%%) times the duration that the pin is equal to pulse_level. If the pin is already equal to pulse_level then timing starts straight away.

> The function will return -2 if there was timeout waiting for condition marked (%) 
above, and -1 if there was timeout during the main measurement, marked (%%) above. The timeout is the same for both cases and given by timeout_us (which is in microseconds).



In [20]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [2]:
%sendtofile config.txt

wifiname       DoESLiverpool
wifipassword   decafbad00
mqttbroker     mqtt.local
boardname      esp32powflash 
watchdogtimeoutseconds  60


Sent 5 lines (138 bytes) to config.txt.


In [3]:
%sendtofile main.py

import machine, time, network, ubinascii
from umqtt.robust import MQTTClient

fconfig = dict(x.split()  for x in open("config.txt"))
wifiname = fconfig["wifiname"]
wifipassword = fconfig["wifipassword"]
mqttbroker = fconfig["mqttbroker"]
boardname = fconfig["boardname"]
watchdogtimeoutseconds = int(fconfig.get("watchdogtimeoutseconds", -1))

pled = machine.Pin(2, machine.Pin.OUT)
pdetector = machine.Pin(15, machine.Pin.IN)

# delays to allow serial line interrupt before wifi or watchdog get put on
pled.value(1)
time.sleep(5)

# from here on the watchdog will reset if there are any crashing out
w = machine.WDT(timeout=watchdogtimeoutseconds*1000) if watchdogtimeoutseconds > 0 else None

si = network.WLAN(network.STA_IF)
si.active(True)
if w is not None:  w.feed()

print("Connecting to", wifiname)
si.connect(wifiname, wifipassword)
while not si.isconnected():
    time.sleep(0.1)
pled.value(0) 
print("connected to wifi")
for i in range(10):
    pled.value(i%2)
    time.sleep_ms(100)
pled.value(1) 
if w is not None:  w.feed()


print("connecting to mqtt", mqttbroker, "as", boardname)
client = MQTTClient(boardname, mqttbroker)
client.connect()
pled.value(0) 
print("connected to mqtt")
for i in range(10):
    pled.value(i%2)
    time.sleep_ms(50)
if w is not None:  w.feed()

# publish startup information 
ipnumber = si.ifconfig()[0]
macaddress = ubinascii.hexlify(si.config('mac'),':').decode().upper()
client.publish(boardname+"/ip", ipnumber, retain=True, qos=1)
client.publish(boardname+"/mac", macaddress, retain=True, qos=1)
resetcause = { machine.PWRON_RESET:"PWRON_RESET", machine.HARD_RESET:"HARD_RESET", machine.WDT_RESET:"WDT_RESET", machine.DEEPSLEEP_RESET:"DEEPSLEEP_RESET", machine.SOFT_RESET:"SOFT_RESET"}\
        .get(machine.reset_cause(), "RESET_%d"%machine.reset_cause())
client.publish(boardname+"/resetcause", resetcause, retain=True, qos=1)
if w is not None:  w.feed()

# final measuring flashes loop
topicnameticks = boardname+"/ticks"
topicnametimeout = boardname+"/timeout"
n, t0 = 0, 0
while True:
    x = machine.time_pulse_us(pdetector, 0, 10000000)//1000   # 10 second timeout
    t = time.ticks_ms()
    dt = t - t0
    if w is not None:  w.feed()
    
    print(x, t, n)
    if x < 0:
        client.publish(topicnametimeout, "%d %d %d" % (dt, x, n))
        continue
        
    if x >= 15 and dt >- 15:
        client.publish(topicnameticks, "%d %d %d" % (dt, x, n))
        t0 = t
        n += 1
        pled.value(n%2) 



Sent 78 lines (2473 bytes) to main.py.
